In [1]:
# Dependencies and set up
import pandas as pd
import numpy as np
import json

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify
from sqlalchemy import inspect

from sqlite3 import connect

# City of Chicago -- Crimes - from 2001 to 2021

In [2]:
# Raw crime table
table = pd.read_csv('Crimes_-_2001_to_Present.csv') 
table.head(2)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.6700,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.7654,"(41.895080471, -87.765400451)"


In [3]:
# table.drop((table['Year'] < 2010).index, inplace=True)

In [4]:
table = table[(table.Year>2010) & (table.Year<2021)]

In [5]:
table.reset_index(inplace=True)

In [6]:
table.head()

,index,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [7]:
# Groupby Primary type
# df = table.groupby(['Primary Type']).count()['Description']
# df

In [8]:
# Dropping columns
table = table.drop(columns=['Case Number', 'Location', 'Updated On', 'Block', 'Community Area', 'Beat', 'District', 'Ward', 'FBI Code', 'X Coordinate', 'Y Coordinate' ])
table.head()

,index,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Year,Latitude,Longitude
0,0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000
1,1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400
2,2,11646166,09/01/2018 12:01:00 AM,0810,THEFT,OVER $500,RESIDENCE,False,True,2018,NaN,NaN
3,3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650
4,4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121


In [9]:
# Dropping NAN from latitude
table = table[table['Latitude'].notna()]
table.head()

,index,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Year,Latitude,Longitude
0,0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000
1,1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400
3,3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650
4,4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121
5,5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,2015,41.744379,-87.658431


In [10]:
table['Month'] = table['Date'].str[0:2].astype(int)
table_db = table.drop(['index'], axis=1)
table_db.head()

,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Year,Latitude,Longitude,Month
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,2015,41.815117,-87.670000,9
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,2015,41.895080,-87.765400,9
3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,2015,41.937406,-87.716650,9
4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,APARTMENT,False,True,2015,41.881903,-87.755121,9
5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,False,False,2015,41.744379,-87.658431,9


# Postgres connection - Chicago crime table

In [11]:
# Remember to change to your Postgres password
rds_connection_string = "postgres:123@localhost:5432/Project_03"

# Adding a connection if none are available
engine = create_engine(f'postgresql://{rds_connection_string}', pool_size=10, max_overflow=20)

In [12]:
table_db.to_sql(name='Chicago_crime', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "Chicago_crime_pkey"
DETAIL:  Key ("ID")=(10224738) already exists.

[SQL: INSERT INTO "Chicago_crime" ("ID", "Date", "IUCR", "Primary Type", "Description", "Location Description", "Arrest", "Domestic", "Year", "Latitude", "Longitude", "Month") VALUES (%(ID)s, %(Date)s, %(IUCR)s, %(Primary Type)s, %(Description)s, %(Location Description)s, %(Arrest)s, %(Domestic)s, %(Year)s, %(Latitude)s, %(Longitude)s, %(Month)s)]
[parameters: ({'ID': 10224738, 'Date': '09/05/2015 01:30:00 PM', 'IUCR': '0486', 'Primary Type': 'BATTERY', 'Description': 'DOMESTIC BATTERY SIMPLE', 'Location Description': 'RESIDENCE', 'Arrest': False, 'Domestic': True, 'Year': 2015, 'Latitude': 41.815117281999996, 'Longitude': -87.669999562, 'Month': 9}, {'ID': 10224739, 'Date': '09/04/2015 11:30:00 AM', 'IUCR': '0870', 'Primary Type': 'THEFT', 'Description': 'POCKET-PICKING', 'Location Description': 'CTA BUS', 'Arrest': False

In [ ]:
c = engine.execute('SELECT * FROM Chicago_crime').fetchall()
print(c[:10])